### Primero se importan las bibliotecas necesarias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import psutil
from collections import defaultdict
import seaborn as sns
# Especiales de RecSys
import surprise
from surprise import accuracy
from surprise import SVD
from surprise import SVDpp
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import cross_validate



### Funciones utiles

In [2]:
def costo_computacional(algoritmo, trainset, imprimir=True):
    proceso = psutil.Process()
    inicio_tiempo = time.time()
    inicio_cpu = proceso.cpu_percent(interval=None)
    inicio_memoria = proceso.memory_info().rss

    algoritmo.fit(trainset)

    fin_tiempo = time.time()
    fin_cpu = proceso.cpu_percent(interval=None)
    fin_memoria = proceso.memory_info().rss

    tiempo = fin_tiempo - inicio_tiempo
    cpu = fin_cpu - inicio_cpu
    memoria = (fin_memoria - inicio_memoria) / (1024 * 1024)

    if imprimir:
        print(f"Tiempo entrenamiento: {tiempo:.2f} segundos")
        print(f"Uso de CPU: {cpu:.2f}%")
        print(f"Uso de memoria: {memoria:.2f} MB")
    return tiempo, cpu, memoria

### Datos a utilizar

In [3]:
train_data = pd.read_csv("datos/train.csv")
validation_data = pd.read_csv("datos/validation.csv")

### Preparación datos

Debido a que se uso la libreria Surprise para todos los modelos (ya que pyRecLab no se podia usar localmente en Mac OS), los datos se prepararon como trainset y testset solo una vez, los cuales luego fueron utilizados por todos los algoritmos enumerados a continuación.

In [4]:
reader = surprise.Reader(rating_scale=(1,5))
trainset = surprise.Dataset.load_from_df(train_data[["user_id", "item_id", "rating"]], reader).build_full_trainset()
testset = list(zip(validation_data["user_id"], validation_data["item_id"], validation_data["rating"]))

### 1. User-based collaborative filtering

In [5]:
myUserKnn = surprise.KNNBasic(k=7, sim_options={'name': 'pearson', 'user_based': True})
tiempo, cpu, memoria =costo_computacional(myUserKnn, trainset, True)
predictions = myUserKnn.test(testset)
rmse_1 = accuracy.rmse(predictions)
mae_1 = accuracy.mae(predictions)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Tiempo entrenamiento: 0.32 segundos
Uso de CPU: 100.30%
Uso de memoria: 257.97 MB
RMSE: 1.1506
MAE:  0.8847


### 2. Item-based collaborative filtering

In [6]:
myItemKnn = surprise.KNNBasic(k=7, sim_options={'name': 'pearson', 'user_based': False})
tiempo, cpu, memoria =costo_computacional(myItemKnn, trainset, True)
predictions = myItemKnn.test(testset)
rmse_2 = accuracy.rmse(predictions)
mae_2 = accuracy.mae(predictions)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Tiempo entrenamiento: 5.66 segundos
Uso de CPU: 65.60%
Uso de memoria: 770.48 MB
RMSE: 1.1511
MAE:  0.8554


### 3. FunkSVD

In [7]:
funksvd = SVD(n_factors = 100, n_epochs = 20, lr_all = 0.005, reg_all = 0.02)
tiempo, cpu, memoria =costo_computacional(funksvd, trainset, True)
predictions = funksvd.test(testset)
rmse_3 = accuracy.rmse(predictions)
mae_3 = accuracy.mae(predictions)

Tiempo entrenamiento: 0.45 segundos
Uso de CPU: 100.10%
Uso de memoria: 18.38 MB
RMSE: 1.0085
MAE:  0.7297


### 4. SVD++ (opcional)

In [8]:
svdpp = SVDpp(n_factors = 20, n_epochs = 20, lr_all = 0.007, reg_all = 0.02)
tiempo, cpu, memoria =costo_computacional(svdpp, trainset, True)
predictions = svdpp.test(testset)
rmse_4 = accuracy.rmse(predictions)
mae_4 = accuracy.mae(predictions)

Tiempo entrenamiento: 0.70 segundos
Uso de CPU: 100.00%
Uso de memoria: 7.84 MB
RMSE: 1.0033
MAE:  0.7105
